# Isolation thermique et dephasage

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation

import pandas as pd
import ipywidgets as ipw

from scipy.interpolate import interp1d
from scipy.linalg import expm
from bokeh.palettes import RdYlBu11
from bokeh.transform import linear_cmap
from bokeh.io import push_notebook, output_notebook, show
from bokeh.layouts import row, column
from bokeh.plotting import figure, curdoc
from bokeh.models import ColumnDataSource, ColorBar, BasicTickFormatter, BasicTicker, FixedTicker, FuncTickFormatter
from bokeh.client import push_session

from ipywidgets import interact
from ipywidgets import IntSlider, interact, FloatSlider

output_notebook()

On modélise un bâtiment et son isolation comme deux sous-systèmes simples dénotés 1 et 2. L’état de chaque sous-système est caractérisé par sa température. Le sous-système 1 représente l’isolation. Le sous-système 2 représente le reste du batiment pour lequel il n’y a pas de transfert de chaleur avec l’environnement. Le transfert irreversible de chaleur entre le sous-système 2 et le sous-système 1 est décrit par la puissance thermique $P_Q^{(21)}(t)$.

Pour le sous-système $i$ de chaleur spécifique constante $C_i$, et de température $T_i$, on a les expréssions suivantes:

\begin{equation}
U_i=C_i T_i \hspace{0.5cm} i=1,2
\end{equation}

Le sous système 1, l'isolant, est soumis à un transfert de chaleur périodique qui correspond à l'oscillation périodique de la température exxtérieure, e.g la variation quotidienne de température en fonction des heures de la journée. Ce transfert de chaleur est décrit par une puissance extérieure P_Q(t) (complexe pour simplifier):

\begin{equation}
P_Q(t)=P_0\exp(i\omega t)
\end{equation}

\begin{equation}
T=\frac{2\pi}{\omega}
\end{equation} 

où $T$ est la période d'oscillation (typiquement une journée). 


Remarque: Bien que la fonction qui décrit le transfert de chaleur (puissance thermique) soit complexe; c'est sa partie réelle qui décrit le phénomène.

## Solution numérique: intégration des équations 
### Cas 1: sans transfert de chaleur
#### Implémentation numérique de la solution pour $\Delta T$:


In [ ]:
t = np.linspace(0, 10, 10000);

# TODO: Set arbitrary initial conditions
# Temperatures
T10 = 0;
T20 = 1;
# problem constants
tau=1;

# TODO: Implement and plot Delta T
DeltaT=0*t;
plt.plot(t, DeltaT );
plt.xlabel('$t$ [a.u]');
plt.ylabel('$\Delta T$ [K]');

#### Résolution numérique de l'évolution des températures $T_1$ et $T_2$

In [ ]:
##############################
# Written by S.Guinchard on  #
#        02/17/22            #
##############################
from array import *
# Paramètres physiques 
# TODO: Play with parameters
# once you have managaed plotting (see below)
T1=10;
T2=20;
kappa = -1;
A     = 1;
C1    = 1;
C2    = 1;
l     = 1;

# Paramètres numériques
tfin   = 5;
nsteps = 10000;
dt=tfin/nsteps;

# Initialisation variables temps et température
output = np.zeros((2,nsteps));
arrayT = [T1,T2];
time   = np.zeros((1,nsteps));
time[0][0] = 0.0;
for i in range(1,nsteps):
    time[0][i] = time[0][i-1]+dt;   


# Method running Euler method till final time
def run():
    t=0.0;
    for i in range(0, nsteps):
        output[0][i] = arrayT[0];
        output[1][i] = arrayT[1];
        
        arrayT[0]= arrayT[0] - kappa*A/(C2*l)*(arrayT[1]-arrayT[0])*dt;
        arrayT[1]= arrayT[1] + kappa*A/(C1*l)*(arrayT[1]-arrayT[0])*dt;
        t+=dt;
        
run();
# TODO : Plot T1, T2 and T2-T1 = Delta T
plt.plot(time[0], replace here ,'b-', label='$T_1$');
plt.plot(time[0], replace here , 'r-', label='$T_2$');
plt.plot(time[0], replace here , 'g-', label='$\Delta T$');
plt.xlabel('$t$ [a.u]');
plt.ylabel('$\Delta T$ [K]');
plt.legend(loc="upper right");

### Cas 2: Avec transfert de chaleur

Dans le cas d'un transfert de chaleur $P_Q$ vers l'extérieur, on a une contribution supplémentaire à l'équation d'évolution de la température $T_1$

In [ ]:
##############################
# Written by S.Guinchard on  #
#        02/17/22            #
##############################
from array import *
################################################
# Paramètres physiques 
# TODO: Play with parameters
# once you have managaed plotting (see below)
T1=10;
T2=20;
kappa = -1;
A     = 1;
C1    = 1;
C2    = 1;
l     = 1;


# Paramètres numériques
# TODO: Play with tfin
tfin =20;
nsteps=100000;
dt=tfin/nsteps;

######## DO NOT modify anything below #########
###############################################

# Initialisation variables temps et température
output = np.zeros((2,nsteps));
arrayT = [T1,T2];
time = np.zeros((1,nsteps));
time[0][0] = 0.0;
for i in range(1,nsteps):
    time[0][i] = time[0][i-1]+dt;   

PQ = np.zeros((1,nsteps));
for i in range(0,nsteps):
    PQ[0][i] = 1/1000*np.cos(np.pi/3*time[0][i]);

# Method running Euler method till final time
def run():
    t=0.0;
    for i in range(0, nsteps):
        output[0][i] = arrayT[0];
        output[1][i] = arrayT[1];
        
        arrayT[0]= arrayT[0] - kappa*A/(C2*l)*(arrayT[1]-arrayT[0])*dt + PQ[0][i]/C1;
        arrayT[1]= arrayT[1] + kappa*A/(C1*l)*(arrayT[1]-arrayT[0])*dt;
        t+=dt;
        
run();
######################################################
# TODO: Plot PQ over t and the temperatures T1 and T2
plt.figure(1)
plt.plot(time[0], , 'k--')
plt.xlabel('$t$ [a.u]');
plt.ylabel('$P_Q$ [J/s]');
plt.ticklabel_format(axis='y', style='sci');

plt.figure(2)
plt.plot(time[0], ,'b-', label= '$T_1$');
plt.plot(time[0], , 'r-', label='$T_2$');
plt.plot(time[0], , 'g-', label='$\Delta T$');
plt.xlabel('$t$ [a.u]');
plt.ylabel('[K]');
plt.legend(loc="upper right");

### 3)Taux de production d'entropie

In [ ]:
##############################
# Written by S.Guinchard on  #
#        02/21/22            #
##############################
from array import *

# Paramètres physiques
# Paramètres physiques 
# TODO: Play with parameters
# once you have managaed plotting (see below)
T1=10;
T2=20;
S1=1;
S2=1;
kappa = -1;
A     = 1;
C1    = 1;
C2    = 1;
l     = 1;


# Paramètres numériques
tfin =10;
nsteps=1000;
dt=tfin/nsteps;

# Initialisation variables temps et température
outputT = np.zeros((2,nsteps));
outputS = np.zeros((2,nsteps));
arrayT  = [T1,T2];
arrayS  = [S1,S2];
PiS     = np.zeros((1,nsteps));
time    = np.zeros((1,nsteps));
time[0][0] = 0.0;
for i in range(1,nsteps):
    time[0][i] = time[0][i-1]+dt;   

PQ = np.zeros((1,nsteps));
for i in range(0,nsteps):
    PQ[0][i] = 1/1000*np.cos(np.pi/3*time[0][i]);

# Method running Euler method till final time
def run():
    t=0.0;
    for i in range(0, nsteps):
        outputT[0][i] = arrayT[0];
        outputT[1][i] = arrayT[1];
        
        outputS[0][i] = arrayS[0];
        outputS[1][i] = arrayS[1];
        
        arrayT[0]= arrayT[0] - kappa*A/(C2*l)*(arrayT[1]-arrayT[0])*dt;
        arrayT[1]= arrayT[1] + kappa*A/(C1*l)*(arrayT[1]-arrayT[0])*dt;
        
        arrayS[0]= arrayS[0] + kappa*A/(l)*(arrayT[1]-arrayT[0])/arrayT[0]*dt 
        arrayS[1]= arrayS[1] - kappa*A/(l)*(arrayT[1]-arrayT[0])/arrayT[0]*dt;
        
        PiS[0][i] = -kappa*A/l*((arrayT[1]-arrayT[0])*(arrayT[1]-arrayT[0]))/(arrayT[1]*arrayT[0]);
        t+=dt;
        
run();
##########################################
# TODO : Plot the entropy production rate
plt.plot(time[0], ,'b-', label='$S_1$');
plt.plot(time[0], , 'r-', label='$S_2$');
plt.plot(time[0], , 'g-', label='$\Pi_S$');
plt.xlabel('$t$ [a.u]');
plt.ylabel('[J/K]');
plt.legend(loc="upper right");